# Construct a custom Environment for Financial Trading

Some examples on the market
* [custom env example](https://colab.research.google.com/github/araffin/rl-tutorial-jnrr19/blob/sb3/5_custom_gym_env.ipynb#scrollTo=RqxatIwPOXe_)
* [StockTradingEnv by Adam King](https://github.com/notadamking/Stock-Trading-Environment)
* [FinRL](https://github.com/AI4Finance-Foundation/FinRL)

Target is to construct a custom Env for pair trading

This env restrict the behaviour of RL learner to pair trading only

In [1]:
import os
import numpy as np
import gymnasium as gym
from gymnasium import spaces
from datetime import date
from sklearn.model_selection import train_test_split

from utils.read2df import read2df

os.makedirs("result/rl-restrict", exist_ok=True)
strategy_txt = f"result/rl-restrict/strategy.txt"
os.remove(strategy_txt) if os.path.exists(strategy_txt) else None

Define data parameters

In [2]:
symbols = ['BTCUSDT', 'ETHUSDT', 'LTCUSDT', 'XMRUSDT', 'BNBUSDT', 'ADAUSDT', 'DOGEUSDT', 'SOLUSDT', 'TRXUSDT']

start_date = '2022-01-01'
trade_date = '2023-01-01'

# freqs = {'1h':60, '2h':120, '4h':240, '6h':360, '8h':480, '12h':720, '1d':1440}
freqs = {'3m':3, '5m':5, '15m':15, '30m':30}

Download data from `binance-public-data`

In [3]:
%%capture
if symbols is None:
    !python binance-public-data/python/download-kline.py -i {" ".join(list(freqs.keys()))} -startDate {start_date} -t spot -skip-daily 1
else:
    !python binance-public-data/python/download-kline.py -s {" ".join(symbols)} -i {" ".join(list(freqs.keys()))} -startDate {start_date} -t spot -skip-daily 1

In [4]:
# dfs = read2df(symbols, freqs)
dfs = read2df(symbols, freqs)

df0 = dfs[0][dfs[0]['tic']=='BTCUSDT'].reset_index(drop=True)
df1 = dfs[0][dfs[0]['tic']=='ETHUSDT'].reset_index(drop=True)

Set data before `trade_data` as training data, after `trade_data` is trade_data

In [5]:
train0 = df0[df0['datetime'] < trade_date]
train1 = df1[df1['datetime'] < trade_date]

test0 = df0[df0['datetime'] >= trade_date]
test1 = df1[df1['datetime'] >= trade_date]

In [6]:
df0[['time', 'close']]

,time,close
0,1597125779999,11850.14
1,1597125959999,11847.86
2,1597126139999,11848.32
3,1597126319999,11840.89
4,1597126499999,11836.65
...,...,...
549477,1696117679999,26966.38
549478,1696117859999,26974.11
549479,1696118039999,26969.55
549480,1696118219999,26964.77


In [7]:
# Don't use custom observation & action spaces
# See the warning on https://gymnasium.farama.org/api/spaces/

'''
class PairTradingActionSpace(gym.Space):
  def __init__(self, low=-1.0, high=1.0, shape=(2, ), dtype=np.float32):
    super().__init__(shape, dtype)
    self.low = low
    self.high = high

  def sample(self):
    action = np.random.uniform(self.low, self.high, self.shape)
    # Normalize the action so that the summation of action[0] and action[1] is within -1 and 1.
    action = action / np.linalg.norm(action)
    return action

  def contains(self, x):
    return np.all(self.low <= x) and np.all(x <= self.high) and np.linalg.norm(x) <= 1.0
'''

'\nclass PairTradingActionSpace(gym.Space):\n  def __init__(self, low=-1.0, high=1.0, shape=(2, ), dtype=np.float32):\n    super().__init__(shape, dtype)\n    self.low = low\n    self.high = high\n\n  def sample(self):\n    action = np.random.uniform(self.low, self.high, self.shape)\n    # Normalize the action so that the summation of action[0] and action[1] is within -1 and 1.\n    action = action / np.linalg.norm(action)\n    return action\n\n  def contains(self, x):\n    return np.all(self.low <= x) and np.all(x <= self.high) and np.linalg.norm(x) <= 1.0\n'

# Define the custom Environment

The behaviour of RL learner is restricted. 

The action is defined as discrete actions -1, 0, 1

-1 means short df0 long df1, 0 means close position, +1 means long df0 short df1

In [8]:
# The lookback period for the observation space
PERIOD = 100 # Only look at the current price
CASH = 10000

class PairTradingEnv(gym.Env):
    metadata = {'render.modes': ['console']}

    # for pair trading, we need to feed in two OHLCV dataframes
    def __init__(self, df0, df1, tc=0.001):
        super().__init__()

        if not df0['time'].equals(df1['time']):
            raise ValueError("Two dataframe must have same time index")

        self.tic0 = df0['tic'].iloc[0]
        self.tic1 = df1['tic'].iloc[0]

        # transaction cost
        self.tc = tc

        # get two datasets
        # self.df0 = df0[['time', 'open', 'high', 'low', 'close', 'volume']]
        # self.df1 = df1[['time', 'open', 'high', 'low', 'close', 'volume']]

        column_feed = ['close']
        self.df0 = df0[column_feed]
        self.df1 = df1[column_feed]

        self.reward_range = (-np.inf, np.inf)

        # We have to start fro 0 according to baseline3
        # 0 means short df0 long df1, 1 means do nothing, 2 means long df0 short df1
        self.action_space = spaces.Discrete(3) # {-1, 0, 1}

        # The data requires to be at least [time, open, high, low, close, volume]
        # Let's assume that we feed in previous 30 period data into the observation_space
        self.observation_space = spaces.Box(low=0.0, high=np.inf, shape=(2*(PERIOD-1)*len(column_feed),), dtype=np.float64)

        # if the length is 35, then the index shall be 0~34
        self.max_steps = len(df0)-1

    def _next_observation(self):
        # The current step is always higher than the PERIOD as defined in the 

        obs_df0 = self.df0.iloc[self.current_step-PERIOD+1: self.current_step]
        obs_df1 = self.df1.iloc[self.current_step-PERIOD+1: self.current_step]

        obs = np.array([obs_df0, obs_df1]).flatten()

        return obs

    def _take_action(self, action):
        # Make it 0 means close position, -1 means short df0 long df1, 1 means long df0 short df1
        # The reason we -1 here is because Baseline requires the action starts with 0
        self.action = action-1

        curr_price0 = self.df0['close'].iloc[self.current_step]
        curr_price1 = self.df1['close'].iloc[self.current_step]

        curr_holding0 = self.holding0
        curr_holding1 = self.holding1

        prev_action = self.holding0 > 0

        # We shouldn't do any action if we are already holding the position
        if prev_action != self.action:

            # evaluate purchasing power 
            max_amount0 = self.net_worth/curr_price0
            max_amount1 = self.net_worth/curr_price1

            # if action is 0, then both holding becomes 0
            # if action is -1, then short df0 long df1 to the max_amount
            # if action is 1, then long df0 short df1 to the max_amount
            self.holding0 = self.action*max_amount0
            self.holding1 = -self.action*max_amount1

            # if action is 0, then clear out based on curr_holding*curr_price
            # if action is -1, then use all the money to sell df0 and buy df1
            # if action is 1, then use all the money to buy df1 and sell df0
            self.cash -= (self.holding0-curr_holding0)*curr_price0 + (self.holding1-curr_holding1)*curr_price1

        # We record the net_worth from previous period to prev_net_worth
        self.prev_net_worth = self.net_worth
        self.net_worth = self.cash + curr_holding0*curr_price0 + curr_holding1*curr_price1

    def step(self, action):
        self._take_action(action)
        self.current_step += 1

        observation = self._next_observation()
        reward = self.net_worth - self.prev_net_worth
        terminated = bool(self.current_step >= self.max_steps)
        truncated = bool(self.net_worth <= 0)
        info = {}

        return observation, reward, terminated, truncated, info

    def reset(self, seed=None):
        np.random.seed(seed)
        
        self.cash = CASH
        self.net_worth = CASH
        self.max_net_worth = CASH
        self.holding0 = 0
        self.holding1 = 0

        self.current_step = np.random.randint(PERIOD, self.max_steps)

        return self._next_observation(), {}
    
    def render(self):
        profit = self.net_worth - CASH

        with open(strategy_txt, "a") as f:
            f.write(f"Current net worth is {self.net_worth}, cash is {self.cash}\n")
            f.write(f"Actions for this step is {self.action}\n")
            f.write(f"Current holding is {self.holding0} of {self.tic0} and {self.holding1} of {self.tic1}\n\n")
        f.close()

## Check with baselin3 `env_checker`

Check if the env meets the requirements of `stable_baseline3`

In [9]:
from stable_baselines3.common.env_checker import check_env

env = PairTradingEnv(train0, train1)
check_env(env)

## Do a test run with random generated actions

In [10]:
import random

env = PairTradingEnv(train0, train1)

obs, _ = env.reset()

print(f"observation_space: {env.observation_space}")
print(f"action_space: {env.action_space}")
print(f"action_space.sample: {env.action_space.sample()}")

n_steps = 5

with open(strategy_txt, "a") as f:
    f.write(f"Start random testing\n")
f.close()

for step in range(n_steps):
    with open(strategy_txt, "a") as f:
        f.write(f"Step {step + 1}\n")
    f.close()
    obs, reward, terminated, truncated, info = env.step(action=random.randint(0, 2))
    done = terminated or truncated
    env.render()
    if done:
        break

observation_space: Box(0.0, inf, (198,), float64)
action_space: Discrete(3)
action_space.sample: 2


## PPO model from stable_baselines3

Train with training data

In [11]:
from stable_baselines3 import PPO

env = PairTradingEnv(train0, train1)

model = PPO("MlpPolicy", env, verbose=0, tensorboard_log="logs")
model.learn(total_timesteps=100000)
model.save("ppo_pairtrading")

## Use the model on Trade data

In [12]:
del model
model = PPO.load("ppo_pairtrading")

In [13]:
env = PairTradingEnv(test0, test1)

with open(strategy_txt, "a") as f:
    f.write(f"Start test trading\n")
f.close()

env.reset()
while True:
    action, _states = model.predict(obs)
    observation, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated
    env.render()
    if terminated:
        print("Test Finished!")
        break
    elif truncated:
        print("bankrupted!")
        break

Test Finished!
